## Análisis de Particle Swarm para una fuerza de ventas de 10 nodos

Particle Swarm es un algoritmo que recibe los siguientes hiperárametros de entrada: 
+ $\alpha$
+ $\beta$ 
+ *número de iteraciones*
+ *número de partículas*

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de particle swarm para un grafo con 10 nodos. Como primer objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 100 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 10 nodos. Se decidió elegir la fuerza de venta **96298** perteneciente a la Ciudad de México para llevar a cabo estas pruebas.

In [1]:
# Librerías
import pandas as pd
import sys
sys.path.append('../')
from src import Utileria as ut
from src.models import particle_swarm as ps

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [2]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(96298))
df_Grafo

,id_origen,id_destino,distancia
0,11078,1009790047,3.1011127017081943
1,11078,1009791566,0.5142577124821974
2,11078,1020088646,0.9248048280652852
3,11078,1020249367,0.47811555481864304
4,11078,1020253076,0.6722824338378225
5,11078,1020300220,0.7785165511359913
6,11078,1020328100,0.9287372777889837
7,11078,1020449326,0.3605529558796124
8,11078,1020451581,0.46907337618604666
9,1009790047,1009791566,3.609300132764086


In [3]:
# Se crea el diccionario de hiper-parámetros que se evaluarán
dict_Hiper_PS = {'Iteraciones': {10,50,100},
              'Particulas': {1, 15,100},
              'Alfa': {.5, 1},
              'Beta': {.5, 1}
              }

**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. En este caso se considerarán 3 valores de *Iteraciones*, 3 valores del *Número de Partículas*, 2 valores de $\alpha$ y 2 de $\beta$; dando lugar a un total de 36 combinaciones. Cada combinación de hiperarámetros se correrá 100 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 100 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [6]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, ps.ParticleSwarm, dict_Hiper_PS, 100)

CPU times: user 1min 18s, sys: 97.3 ms, total: 1min 18s
Wall time: 1min 18s


In [7]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 0.5, 'Beta': 0.5}",9.060,11.696,1/100
1,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 0.5, 'Beta': 1.0}",9.072,11.798,1/100
2,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 1.0, 'Beta': 0.5}",9.039,11.780,1/100
3,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 1.0, 'Beta': 1.0}",8.993,11.697,1/100
4,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 0.5, 'Beta': 0.5}",8.458,8.801,46/100
5,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 0.5, 'Beta': 1.0}",8.458,8.772,27/100
6,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 1.0, 'Beta': 0.5}",8.458,8.772,54/100
7,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 1.0, 'Beta': 1.0}",8.458,9.101,12/100
8,"{'Iteraciones': 100, 'Particulas': 15, 'Alfa': 0.5, 'Beta': 0.5}",8.458,9.044,7/100
9,"{'Iteraciones': 100, 'Particulas': 15, 'Alfa': 0.5, 'Beta': 1.0}",8.458,9.057,6/100


In [5]:
# Mínimo de 'Distancia mínima (km)'
dist_min = df_Resultado.columns[1]
min(df_Resultado[dist_min])

Decimal('8.458')

In [6]:
# Máximo de 'Distancia mínima (km)'
dist_min = df_Resultado.columns[1]
max(df_Resultado[dist_min])

Decimal('9.469')

In [7]:
# Mínimo de 'Distancia máxima (km)'
dist_max = df_Resultado.columns[2]
min(df_Resultado[dist_max])

Decimal('8.674')

In [8]:
# Máximo de 'Distancia máxima (km)'
dist_max = df_Resultado.columns[2]
max(df_Resultado[dist_max])

Decimal('11.876')

**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de los hiperparámetros elegidos. 
+ Con respecto al número de iteraciones, $100$ iteraciones fueron elegidas con base en pruebas realizadas en la literatura [](), tomando este número como base, se determinó hacer pruebas considerando la mitad de iteraciones $50$; y finalmente para tener un valor extremo contra el cuál comparar se eligió disminuir el número de iteraciones incial en un $90\%$, dando lugar a $10$ iteraciones.
+ Sobre el número de partículas, la primera idea fue considerar un número de partículas similar al número de nodos, por lo que elegimos $15$; y tomar un número de partículas considerablemente mayor $100$ y uno considerablemente menor $1$. En este caso, al tener únicamente $10$ nodos no es posible tomar un número menor de partículas extremo; sin embargo los resultados obtenidos con $1$ partícula aportan resultados muy relevantes para el análisis.
+ Puesto que $\alpha$ y $\beta$ representan probabilidades, los valores que pueden tomar están entre $0$ y $1$. De manera análoga a los otros parámetros, se consideraron valores bajos $0$, medios $0.5$ y altos $1$.

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron $36$ pruebas, es decir, se tuvieron $36$ combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió $100$ veces, dando un total de $3,600$ corridas. En las $100$ corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:
+ La distancia mínima obtenida varia entre $8.458$ km y $9.469$ km, mientras que la distancia máxima varía entre $8.674$ km y $11.876$ km. Apesar de utilizar los mismos hiperparámetros, el algoritmo puede dar resultados distintos en cada corrida.
+ Al considerar únicamente $1$ partícula y variar el resto de los hiperparámetros, se observa que la frecuencia relativa de la distancia mínima en cada una de las pruebas $100$ corridas por prueba es de 1%.
+ En general podemos observar que al aumentar el número de partículas la frecuencia relativa de la distancia mínima aumenta.
+ Al igual que en las pruebas con $6$ nodos, se idenficó que para los casos en los que $\alpha$ y $\beta$ son $1$, se obtienen las frecuencias relativas de la distancia mínima mas pequeñas.
+ En la gran mayoría se observa que los valores de los parámetros $\alpha$ y $\beta$ que mejores resultados dan son $1$ y $0.5$ respectivamente, las frecuencias relativas de la distancia mínima son las mayores y ademas la diferencia entre las distancia mínima y la distancia máxima son las mas pequeñas, lo que nos dice que los resultados del algoritmo varian menos y por ende se encuentran mas cercanos al mínimo.
+ Obervando los resultados podemos darnos ver que el parámetro que tiene un mayor impacto para obtener "buenos resultados" es el de número de partículas. En los casos en los que el número de particulas es $100$ son en los que se obtienen las frecuencias relativas de la distancia mínima mas altas.

### 2. Análisis de rutas

Los mejores hiperparámetros para una fuerza de ventas de 10 nodos son los siguientes:
+ Número de iteraciones: 100
+ Número de partículas: 100
+ $\alpha = 1$
+ $\beta = 0.5$

Fijando los valores anteriores, se correrá el algoritmo de Particle Swarm 100 veces para la fuerza de ventas con 10 nodos y se llevará a cabo un análisis sobre las distintas rutas que arroja el algoritmo, así como la distancia mínima obtenida para tales rutas.

In [11]:
# Definición de dataframe donde se almacenarán las rutas
rutas = pd.DataFrame(index=range(100),columns=['Distancia', 'Ruta'])

# Definición de hiperparámetros
dict_Hiper = {'Iteraciones': 100,
              'Particulas': 100,
              'Alfa': 1,
              'Beta': .5
              }

In [12]:
%%time

# 100 ejecuciones de particle swarm para una fuerza de ventas con 10 nodos,
# considerando los mejores hiperparámetros

for corrida in range(100):
    
    PS = ps.ParticleSwarm(df_Grafo,dict_Hiper)
    PS.Ejecutar()
    
    min_distancia = round(PS.nbr_MejorCosto,3)
    mejor_ruta = ut.convert(PS.lst_MejorCamino)
    
    rutas.Distancia[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta

CPU times: user 10 s, sys: 37.8 ms, total: 10 s
Wall time: 10.1 s


In [17]:
# Primeras 10 rutas obtenidas
rutas.head(10)

,Distancia,Ruta
0,8.458,11078-1020451581-1009791566-1020328100-1020088...
1,8.559,11078-1020451581-1009791566-1020253076-1020328...
2,8.458,11078-1009790047-1020449326-1020249367-1020253...
3,8.458,11078-1020451581-1009791566-1020328100-1020088...
4,8.458,11078-1009790047-1020449326-1020249367-1020253...
5,8.559,11078-1020451581-1009791566-1020253076-1020328...
6,8.459,11078-1020449326-1020249367-1020253076-1020300...
7,8.559,11078-1009790047-1020449326-1020249367-1020300...
8,8.559,11078-1009790047-1020449326-1020249367-1020300...
9,8.458,11078-1020451581-1009791566-1020328100-1020088...


In [25]:
# Descripción de las rutas obtenidas
descripcion = rutas.groupby('Ruta').describe()
count = descripcion.columns[0]
descripcion.sort_values(count, ascending=False)

Distancia                \
                                                       count unique    top   
Ruta                                                                         
11078-1020451581-1009791566-1020328100-10200886...        29      1  8.458   
11078-1009790047-1020449326-1020249367-10202530...        26      1  8.458   
11078-1020451581-1009791566-1020253076-10203281...        14      1  8.559   
11078-1009790047-1020449326-1020249367-10203002...        10      1  8.559   
11078-1020449326-1020249367-1020300220-10200886...         4      1  8.559   
11078-1020449326-1020249367-1020253076-10203002...         4      1  8.459   
11078-1009790047-1020451581-1009791566-10202530...         2      1  8.559   
11078-1009790047-1020451581-1009791566-10203281...         2      1  8.459   
11078-1009790047-1020449326-1020249367-10200886...         1      1  8.656   
11078-1009790047-1020449326-1020300220-10200886...         1      1  8.674   
11078-1009790047-1020449326-1020249367-10202530...         1      1  8.576   
11078-1020449326-1020451581-1009791566-10203281...         1      1  8.591   
11078-1020451581-1009791566-1020088646-10203281...         1      1  8.679   
11078-1020451581-1009791566-1020249367-10203002...         1      1  8.676   
11078-1020451581-1009791566-1020253076-10203002...         1      1  8.656   
11078-1009790047-1020449326-1020249367-10202530...         1      1  8.566   
11078-1009790047-1020249367-1020300220-10200886...         1      1  8.692   

                                                         
                                                   freq  
Ruta                                                     
11078-1020451581-1009791566-1020328100-10200886...   29  
11078-1009790047-1020449326-1020249367-10202530...   26  
11078-1020451581-1009791566-1020253076-10203281...   14  
11078-1009790047-1020449326-1020249367-10203002...   10  
11078-1020449326-1020249367-1020300220-10200886...    4  
11078-1020449326-1020249367-1020253076-10203002...    4  
11078-1009790047-1020451581-1009791566-10202530...    2  
11078-1009790047-1020451581-1009791566-10203281...    2  
11078-1009790047-1020449326-1020249367-10200886...    1  
11078-1009790047-1020449326-1020300220-10200886...    1  
11078-1009790047-1020449326-1020249367-10202530...    1  
11078-1020449326-1020451581-1009791566-10203281...    1  
11078-1020451581-1009791566-1020088646-10203281...    1  
11078-1020451581-1009791566-1020249367-10203002...    1  
11078-1020451581-1009791566-1020253076-10203002...    1  
11078-1009790047-1020449326-1020249367-10202530...    1  
11078-1009790047-1020249367-1020300220-10200886...    1

En las $100$ veces que se ejecutó el algoritmo se obtuvieron $17$ rutas distintas; $2$ de ellas con la misma distancia mínima de $8.458$ km. En el $55\%$ de las ejecuciones se obtuvo la menor distancia.

Este resultado demuestra que el algoritmo de Particle Swarm puede arrojar rutas distintas con el mismo costo (distancia mínima). Por otro lado también es posible que arroje otras rutas cuya distancia no es la mínima distancia obtenida durante todas las simulaciones.

### Conclusiones:
+ Al igual que en las pruebas con 6 nodos los resultados del algoritmo pueden variar en cada corrida, aún cuando se utilicen los mismos hiperpárametros.
+ Aumentar el número de partículas nos da una mayor probabilidad de encontrar el mínimo.
+ Los mejores hiperparámetros fueron los mismos que para la prueba de 6  nodos.
+ Considerando las combinaciones de parámetros que ejecutamos para esta prueba podemos decir que darle un mayor peso a alfa que a beta para este problema nos garantiza mejores resultados.
+ Para un mayor número de nodos, obtenemos un mayor número de rutas distintas.